In [63]:
import tensorflow as tf
import soundfile as sf
import IPython.display as ipd
import pyaudio
import numpy as np

from keras.models import Sequential
from keras.layers import InputLayer, Conv1D, Dense, Flatten, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from efficientnet_lite import EfficientNetLiteB0
from IPython.display import clear_output
from keras.utils import to_categorical

In [71]:
model = Sequential()

model.add(InputLayer([16000, 1]))
model.add(Conv1D(1, 1000, 1))
model.add(Activation('relu'))
model.add(Conv1D(1, 1000, 1))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(2))

early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=3, restore_best_weights=True)
model.compile(optimizer=Adam(lr=1e-4),loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_30 (Conv1D)          (None, 15001, 1)          1001      
                                                                 
 activation_26 (Activation)  (None, 15001, 1)          0         
                                                                 
 conv1d_31 (Conv1D)          (None, 14002, 1)          1001      
                                                                 
 activation_27 (Activation)  (None, 14002, 1)          0         
                                                                 
 flatten_6 (Flatten)         (None, 14002)             0         
                                                                 
 dense_13 (Dense)            (None, 2)                 28006     
                                                                 
Total params: 30,008
Trainable params: 30,008
Non-tra

C:\Users\Jerry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [3]:
import sounddevice as sd
import wavio as wv
from scipy.io.wavfile import write

In [5]:
fs = 44100
duration = 2 # Waiting time in seconds
recording = sd.rec(int(duration * fs), samplerate=fs, channels=2)

sd.wait() # Wait for the recording to finish

dir = r"C:\Users\Jerry\OneDrive\Documents\Imperial\3rd year\3rd year Group project\Train data\Voices\owner 1" # Directory folder to store user voice data
write(dir + "\0.wav", fs, recording)

In [ ]:
window_size = 16000 # number of data points to read at a time
fs = 44100

p = pyaudio.PyAudio() # start the PyAudio class
stream=p.open(format=pyaudio.paInt16,channels=1, rate = fs, input=True, frames_per_buffer = window_size) #uses default input device

while True:
    data = np.fromstring(stream.read(window_size) ,dtype = np.int16)[:, None]
    score_raw = model.predict(data[None])[0]
    score = tf.math.argmax(score_raw)
    print(score)
    
    if score == 1:
        print('Hi boss!')
    elif score == 0:
        print('Anyone there?')

In [68]:
dir = r'C:\Users\Jerry\OneDrive\Documents\Imperial\3rd year\3rd year Group project\Train data\Voices' # Directory folder which stores all the voice data
data = tf.keras.utils.audio_dataset_from_directory(dir, output_sequence_length = 16000, label_mode = 'categorical', batch_size = 62)

Found 62 files belonging to 2 classes.


In [78]:
for audios, labels in data.take(1):  # Takes the 1st batch; in our case there's only 1 batch
    train_X = audios.numpy()
    labels = labels.numpy()
train_X = tf.math.reduce_sum(train_X, axis = -1)[..., None]
print(train_X.shape)

(62, 16000, 1)


In [80]:
model.fit(train_X, labels, epochs=3, batch_size = 1)

Epoch 1/3
62/62 [==============================] - 7s 116ms/step - loss: 1.1921e-07 - categorical_accuracy: 0.4839
Epoch 2/3
62/62 [==============================] - 7s 113ms/step - loss: 1.1921e-07 - categorical_accuracy: 0.4839
Epoch 3/3
62/62 [==============================] - 7s 115ms/step - loss: 1.1921e-07 - categorical_accuracy: 0.4839
